## Lectura de los datos

#### Librerias

In [1]:
import pandas as pd
import numpy as np
import keras_metrics
import tensorflow as tf
import shutil, os



from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers, layers
from tensorflow.keras import utils as k
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing import image_dataset_from_directory


In [2]:

file= 'metadata.csv'
dt=pd.read_csv(file, sep=",")

dt.head(1)

,patientid,offset,sex,age,finding,survival,intubated,intubation_present,went_icu,in_icu,...,date,location,folder,filename,doi,url,license,clinical_notes,other_notes,Unnamed: 28
0,2,0.0,M,65.0,COVID-19,Y,NaN,NaN,NaN,NaN,...,"January 22, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-a-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN


### Preprocesamiento

Para que la funcion de keras: <i>image_dataset_from_directory</i> funcione para poder entrenar como queremos, separamos las fotos en dos capetas, una para covid y otra para el resto de enfermedades.

##### Creamos los directorios

In [3]:
carpeta = os.getcwd()+os.sep+'images'

try:
    os.stat(carpeta+os.sep+'covid')
except:
    os.mkdir(carpeta+os.sep+'covid')
    
try:
    os.stat(carpeta+os.sep+'otro')
except:
    os.mkdir(carpeta+os.sep+'otro')

##### Movemos las imagenes

In [4]:
covid=[]
otro=[]

foto=''
destino=''
ruta=carpeta+destino
for i,r in dt.iterrows():
    foto=r['filename']
    if r['finding'] == 'COVID-19':
        covid.append(r)
        destino='covid'
    else:
        otro.append(r)
        destino='otro'
    if os.path.exists(ruta+os.sep+foto):
        shutil.move(ruta+os.sep+foto,ruta+os.sep+destino)
    #print(ruta+os.sep+foto)
    
        
print("Co: ",len(covid) )
print("Otro: ",len(otro) )
print("Proporcion de Covid: ", len(covid)/(len(covid)+len(otro))*100)
    


Co:  296
Otro:  76
Proporcion de Covid:  79.56989247311827


##### Guardamos las imagenes en memoria

In [8]:


img_fit= image_dataset_from_directory(directory="images", image_size=(512,512), subset="training", validation_split=0.2, shuffle=False,
                                      labels='inferred')
img_val=image_dataset_from_directory(directory="images", image_size=(512,512), subset="validation", validation_split=0.2, shuffle=False,
                                      label_mode=None)

#PARA MEJORAR EL RENDIMIENTO
AUTOTUNE = tf.data.AUTOTUNE
img_fit=img_fit.cache().prefetch(buffer_size=AUTOTUNE)
img_val=img_val.cache().prefetch(buffer_size=AUTOTUNE)

Normalizar= layers.experimental.preprocessing.Rescaling(1./255, input_shape=(512, 512, 3))


Found 351 files belonging to 2 classes.
Using 281 files for training.
Found 351 files belonging to 2 classes.
Using 70 files for validation.


### CREACION DEL MODELO

In [6]:
model= Sequential([
    
    Normalizar,
    layers.Conv2D(filters=16,kernel_size=3, strides=1,activation='relu'),
    layers.MaxPooling2D(pool_size=5),
    layers.Conv2D(filters=32,kernel_size=3, strides=1,activation='relu'),
    layers.MaxPooling2D(pool_size=5),
    layers.Conv2D(filters=64,kernel_size=3, strides=1,activation='relu'),
    layers.MaxPooling2D(pool_size=5),
    layers.Flatten(),
    Dense(256,'relu'),
    Dense(1,'sigmoid')

])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling (Rescaling)        (None, 512, 512, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 510, 510, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 102, 102, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 100, 100, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 20, 20, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 18, 18, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 3, 3, 64)          0

### ENTRENAMIENTO

In [9]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(img_fit, validation_data= img_val, epochs= 10)

Epoch 1/10
9/9 [==============================] - 33s 3s/step - loss: 0.0000e+00 - accuracy: 0.9957 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/10
9/9 [==============================] - 23s 3s/step - loss: 0.0000e+00 - accuracy: 0.9957 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 3/10
9/9 [==============================] - 23s 3s/step - loss: 0.0000e+00 - accuracy: 0.9957 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 4/10
9/9 [==============================] - 22s 2s/step - loss: 0.0000e+00 - accuracy: 0.9957 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 5/10
9/9 [==============================] - 23s 3s/step - loss: 0.0000e+00 - accuracy: 0.9957 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 6/10
9/9 [==============================] - 23s 3s/step - loss: 0.0000e+00 - accuracy: 0.9957 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 7/10
9/9 [==============================] - 23s 3s/step - loss: 0.0000e+00 - accuracy: 0.9957 

In [11]:
model.history['accuracy']

TypeError: 'History' object is not subscriptable